In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

import logging

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-139588
aml-quickstarts-139588
southcentralus
9e65f93e-bdd8-437b-b1e8-0647cd6098f7


In [2]:
experiment_name = 'azure-nd-project-capstone-hyperdrive'
project_folder = './automl-run-capstone-project'

experiment = Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cluster-nd-cs"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4, min_nodes=1)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from register_ds import get_cleaned_dataset

ds_cleaned = get_cleaned_dataset(ws)


## Hyperdrive run for regression

In [7]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform # supported by RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice # supported by RandomParameterSampling
import os
import shutil

from azureml.core import ScriptRunConfig
from azureml.core import Environment

# Parameter sampler for the HyperDrive
ps = RandomParameterSampling(
    {
        '--learning_rate': uniform(0.01, 0.3),# Contribution of each tree: uniform discribution 
        '--n_estimators': choice(100, 150, 200, 250, 300, 350), # Number of learners
    }
)

# This policy compares the value (Y + Y * 0.2) to "best current score", and if smaller, cancels the run.
policy = BanditPolicy(slack_factor=0.01, delay_evaluation = 50)


script_folder = './scripts'

# Create data consumption config for the Run, Dataset is consumed in 'direct' mode, 
dataset_consumption_cfg = ds_cleaned.as_named_input('dataset') 

env = Environment.get(workspace=ws, name="AzureML-AutoML")

#ScriptRunConfig
run_config = ScriptRunConfig(source_directory=script_folder,
                             script='hyperdrive_train.py',
                             compute_target=compute_target, environment=env,
                             arguments=['--input-data', dataset_consumption_cfg])
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     policy=policy,
                                     run_config=run_config,
                                      #The name of the primary metric reported by the experiment runs.
                                     primary_metric_name='r2_score',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 20,
                                     max_duration_minutes=40,
                                     max_concurrent_runs=3) # 4 nodes


In [8]:
hyperdrive_run = experiment.submit(hyperdrive_config)

In [9]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
# Get the best run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
#print(best_run.get_details()['runDefinition']['arguments'])
best_run.get_details()

{'runId': 'HD_25b47e91-755d-4611-a0af-e16b8c44a6e2_0',
 'target': 'cluster-nd-cs',
 'status': 'Completed',
 'startTimeUtc': '2021-02-27T23:29:01.201518Z',
 'endTimeUtc': '2021-02-27T23:31:03.159695Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'ea067e67-79fc-4bf5-9935-6b0a2514cde4',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'ed50b071-0ae9-4658-8405-d907aa702bf4'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'dataset', 'mechanism': 'Direct'}}, {'dataset': {'id': 'ed50b071-0ae9-4658-8405-d907aa702bf4'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'hyperdrive_train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--input-data',
   'DatasetConsumptionConfig:dataset',
   '--learning_rate',
   '0.18891278013532103',
   '--n_estimators',
   '150'],
  'sourceDir

In [11]:
# Register the folder (and all files in it) as a model named 'best-model-openfoodfacts-hd' under the workspace,
# the name of the model 'model.joblib' is fixed in hyperdrive_train.py
model_hd = best_run.register_model(model_name='best-model-machine-cpu-hd', model_path='outputs/model.joblib')

# Deploy the model

In [12]:
from azureml.core.webservice import Webservice, AciWebservice
from azureml.core.model import Model

print("Prepare ACI deployment configuration")
# Enable application insights
config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                            memory_gb = 1,
                                            enable_app_insights=True,
                                            auth_enabled=True)


Prepare ACI deployment configuration


In [13]:
from azureml.core.model import InferenceConfig


# creating inference configuration with the scoring file generated by AutoML run
inference_config = InferenceConfig(entry_script='./scripts/score.py',
                                   environment=env)

In [15]:
print("Deploy the model to ACI")
# deploying to ACI using default environment and the generated  scoring script
service_name_hd = 'machine-cpu-service-hd'
service_hd = Model.deploy(workspace=ws, name = service_name_hd, models=[model_hd],
                       overwrite=True, deployment_config=config, inference_config=inference_config)


service_hd.wait_for_deployment(show_output = True)
print(service_hd.state)

Deploy the model to ACI
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [60]:
print(service_hd.get_logs())

2021-02-27T23:54:51,934375200+00:00 - rsyslog/run 
2021-02-27T23:54:51,937321000+00:00 - gunicorn/run 
2021-02-27T23:54:51,947553200+00:00 - nginx/run 
2021-02-27T23:54:51,969567000+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

# Invoke the model
Sending a request to the web service deployed.

In [65]:
import requests
import json

#score using ONNX model
headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}

if service_hd.auth_enabled:
    headers['Authorization'] = 'Bearer '+ service_hd.get_keys()[0]
elif service_hd.token_auth_enabled:
    headers['Authorization'] = 'Bearer '+ service_hd.get_token()[0]

scoring_uri = service_hd.scoring_uri
#print(service_hd)

input_payload = json.dumps({
    "data":
input_payload = json.dumps({
    "data":
         [{"MYCT": 29,"MMIN": 8000,"MMAX": 32000,"CACH": 32,"CHMIN": 8,"CHMAX": 32,"PRP": 208,"vendor_adviser": 0,
           "vendor_amdahl": 1,"vendor_apollo": 0,"vendor_basf": 0,"vendor_bti": 0,"vendor_burroughs": 0,
           "vendor_c.r.d": 0,"vendor_cambex": 0,"vendor_cdc": 0,"vendor_dec": 0,"vendor_dg": 0,"vendor_formation": 0,
           "vendor_four-phase": 0,"vendor_gould": 0,"vendor_harris": 0,"vendor_honeywell": 0,"vendor_hp": 0,"vendor_ibm": 0,
           "vendor_ipl": 0,"vendor_magnuson": 0,"vendor_microdata": 0,"vendor_nas": 0,"vendor_ncr": 0,"vendor_nixdorf": 0,
           "vendor_perkin-elmer": 0,"vendor_prime": 0,"vendor_siemens": 0,"vendor_sperry": 0,"vendor_sratus": 0,        
           "vendor_wang": 0}],
    'method': 'predict' 
})

response = requests.post(
        scoring_uri, data=input_payload, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

200
0:00:00.072580
[233.17841745643233]


In [66]:
service_hd.delete()

In [67]:
compute_target.delete()